In [4]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv(r"..\preprocessing\county_variables.csv")
num_lag = 2 # number of lag features to predict from

data_cols = [col for col in data.columns if col not in ['Date', 'Hour']]
for lag in range(1, num_lag+1):
    for col in data_cols:
        data[f'{col}_lag{lag}'] = data[col].shift(lag)
        #data[f'{col}_lag{lag}_diff'] = data[col] - data[f'{col}_lag{lag}']
        data[f'{col}_lag{lag}_diff_scaled'] = (data[col] - data[f'{col}_lag{lag}'])/lag

num_targets = 1 # number of forward features to predict
for target in range(1, num_targets+1):
    for col in data_cols:
        data[f'{col}_target{target}'] = data[col].shift(-target)
        data[f'{col}_target{target}_diff'] = data[f'{col}_target{target}'] - data[col]

# time input features
data['Date'] = pd.to_datetime(data['Date'])
data['doy'] = data['Date'].dt.day_of_year
data['years_since_2000'] = (data['Date'] - pd.to_datetime('2000-01-01')) / pd.Timedelta(days=365.25)
#data['cos_doy'] = np.cos(2*np.pi*data['doy']/365.25)
#data['sin_doy'] = np.sin(2*np.pi*data['doy']/365.25)

data = data.iloc[num_lag:-num_targets]
data.to_csv('features.csv', index=False)

total_data_points = len(data.index)
training_data_pct = 0.7
training_data = data.iloc[:int(training_data_pct * total_data_points)]
testing_data = data.iloc[int(training_data_pct * total_data_points):]

# training_data.to_csv('training.csv', index=False)
# testing_data.to_csv('testing.csv', index=False)

In [24]:
input_features = [col for col in training_data.columns if ('target' not in col and col != 'Date')]
output_features = [col for col in training_data.columns if ('target' in col and 'diff' in col)]

In [26]:
print(len(input_features), len(output_features))

585 116
